## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:37,  8.00it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:43<00:00, 173.30it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Time4Stream,"Napoli, Campania",Time4Stream is a service company operating in ...,2015-02-26 10:33:33,275,1247,12449,False,2020-07-27 07:17:20,Post match Interview with Luca Gotti and Walte...,None,Twitter for Android,False
0,Bradley...,Hampshire,"Creative geek, data-analyst, student of Zen, t...",2012-04-11 18:36:02,140,201,788,False,2020-07-27 07:17:19,"Why continued hand-hygiene, masks and social d...",[covid19],Twitter Web App,False
0,Rajesh Kshitij,hyderabad,News 18 Network,2010-07-25 10:45:13,504,980,4613,False,2020-07-27 07:17:18,"#Puducherry - 86 new #COVID19 positive cases, ...","[Puducherry, COVID19]",TweetDeck,False
0,ANA-MPA news,"Athens, Greece",Athens News Agency - Macedonian Press Agency (...,2013-02-02 00:15:07,9453,3484,36,True,2020-07-27 07:17:18,Instructions for passengers and crews on cruis...,[Greece],Twitter Web App,False
0,Rosebank Killarney,Johannesburg,All your local news supplied by the Rosebank K...,2010-08-31 08:55:43,5508,1313,348,False,2020-07-27 07:17:18,#Covid19 11 233 new cases as 265 077 recover f...,[Covid19],Fabrik.fm,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (24676, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 24676 all tweets: 32176


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (32176, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)